In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator


https://www.kaggle.com/datasets/tanay001/nseindia-futures-options-daily/data

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.config('spark.driver.memory', '10g').appName('Future and Option Data Analysis').getOrCreate()


/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [5]:

# Load your dataset
data = spark.read.csv("fobhav.csv", header=True, inferSchema=True)

## Filter rows where INSTRUMENT is FUTIDX
#futures_data = data.filter(data['INSTRUMENT'] == 'FUTIDX')

## Show the filtered data
#futures_data.show()




In [12]:
import numpy as np
all_options = np.array(data.filter(data['INSTRUMENT'] == 'OPTSTK').select('SYMBOL').distinct().collect()).tolist()
#print all the different stock names
all_option_1d_array = np.reshape(all_options,(1,-1))
all_option_1d_array = all_option_1d_array[0]
print(all_option_1d_array)

['GODREJCP' 'LT' 'ICICIBANK' 'L&TFH' 'IDEA' 'TITAN' 'MINDTREE'
 'TORNTPOWER' 'SRTRANSFIN' 'IBULHSGFIN' 'BOSCHLTD' 'MANAPPURAM' 'PETRONET'
 'PFC' 'DIVISLAB' 'SAIL' 'EXIDEIND' 'HDFCBANK' 'EQUITAS' 'CIPLA'
 'AUROPHARMA' 'MFSL' 'CENTURYTEX' 'BHARATFORG' 'MRF' 'HINDALCO' 'SBILIFE'
 'ZEEL' 'ACC' 'BAJAJ-AUTO' 'BALKRISIND' 'INFY' 'BATAINDIA' 'M&M' 'IGL'
 'POWERGRID' 'ONGC' 'PNB' 'ASHOKLEY' 'NATIONALUM' 'ADANIPORTS'
 'APOLLOHOSP' 'HDFC' 'M&MFIN' 'AMBUJACEM' 'CUMMINSIND' 'UBL' 'RAMCOCEM'
 'BAJFINANCE' 'SIEMENS' 'VOLTAS' 'SRF' 'HAVELLS' 'BPCL' 'TATACONSUM'
 'EICHERMOT' 'HINDPETRO' 'HINDUNILVR' 'ASIANPAINT' 'MOTHERSUMI'
 'MUTHOOTFIN' 'NAUKRI' 'BHEL' 'NCC' 'NTPC' 'RELIANCE' 'NMDC' 'TATAPOWER'
 'COFORGE' 'TVSMOTOR' 'DABUR' 'DLF' 'MARUTI' 'UPL' 'LICHSGFIN' 'NESTLEIND'
 'TECHM' 'CONCOR' 'UJJIVAN' 'INDIGO' 'MCDOWELL-N' 'SUNPHARMA' 'COLPAL'
 'MARICO' 'PVR' 'BHARTIARTL' 'PAGEIND' 'COALINDIA' 'ADANIENT' 'TORNTPHARM'
 'RECLTD' 'TCS' 'CHOLAFIN' 'RBLBANK' 'INDUSINDBK' 'BANDHANBNK'
 'FEDERALBNK' 'HDFCLIFE' 'G

In [13]:
option_data_LT = data.filter((data['SYMBOL'] == 'LT') & (data['INSTRUMENT'] == 'OPTSTK')).collect()

AttributeError: 'list' object has no attribute 'show'

In [16]:
len(option_data_LT)

633054

In [17]:
type(option_data_LT)

list

In [18]:
import csv
header = ['Column1', 'Column2', 'Column3']
with open('lt_option.csv','w') as f:
    writer = csv.writer(f)
    writer.writerows(option_data_LT)
    

In [10]:
option_data_LT.count()

643420

In [4]:
for name in all_option_1d_array

94010

In [5]:
from pyspark.sql.functions import col

# Assuming futures_data is your DataFrame
futures_data = futures_data.withColumn("SETTLE_PR", col("SETTLE_PR").cast("double"))


In [6]:
futures_data.printSchema()

root
 |-- INSTRUMENT: string (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- EXPIRY_DT: string (nullable = true)
 |-- STRIKE_PR: string (nullable = true)
 |-- OPTION_TYP: string (nullable = true)
 |-- OPEN: string (nullable = true)
 |-- HIGH: string (nullable = true)
 |-- LOW: string (nullable = true)
 |-- CLOSE: string (nullable = true)
 |-- SETTLE_PR: double (nullable = true)
 |-- CONTRACTS: string (nullable = true)
 |-- VAL_INLAKH: string (nullable = true)
 |-- OPEN_INT: string (nullable = true)
 |-- CHG_IN_OI: string (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- Unnamed: 15: string (nullable = true)



##### Now clean the FUTIDX data frame where the option type is xx

In [7]:
futures_data.show(vertical=True)

-RECORD 0------------------
 INSTRUMENT  | FUTIDX      
 SYMBOL      | BANKNIFTY   
 EXPIRY_DT   | 24-Sep-2020 
 STRIKE_PR   | 0.0         
 OPTION_TYP  | XX          
 OPEN        | 24900.05    
 HIGH        | 25270.0     
 LOW         | 23371.0     
 CLOSE       | 23743.0     
 SETTLE_PR   | 23743.0     
 CONTRACTS   | 504222      
 VAL_INLAKH  | 3051593.27  
 OPEN_INT    | 1432850     
 CHG_IN_OI   | -54500      
 TIMESTAMP   | 31-AUG-2020 
 Unnamed: 15 | null        
-RECORD 1------------------
 INSTRUMENT  | FUTIDX      
 SYMBOL      | BANKNIFTY   
 EXPIRY_DT   | 29-Oct-2020 
 STRIKE_PR   | 0.0         
 OPTION_TYP  | XX          
 OPEN        | 24937.55    
 HIGH        | 25315.9     
 LOW         | 23391.9     
 CLOSE       | 23770.1     
 SETTLE_PR   | 23770.1     
 CONTRACTS   | 10630       
 VAL_INLAKH  | 64526.57    
 OPEN_INT    | 91375       
 CHG_IN_OI   | 19325       
 TIMESTAMP   | 31-AUG-2020 
 Unnamed: 15 | null        
-RECORD 2------------------
 INSTRUMENT  | FUTID

In [8]:
futures_data.createGlobalTempView('FUTIDX')

In [9]:
# futures_data_unique = spark.sql('SELECT DISTINCT TIMESTAMP, OPEN, HIGH, LOW, CLOSE FROM global_temp.FUTIDX')

In [ ]:
['INSTRUMENT','SYMBOL','EXPIRY_DT',]

In [10]:
futures_data.show()

+----------+---------+-----------+---------+----------+--------+--------+--------+--------+---------+---------+----------+--------+---------+-----------+-----------+
|INSTRUMENT|   SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|    OPEN|    HIGH|     LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|Unnamed: 15|
+----------+---------+-----------+---------+----------+--------+--------+--------+--------+---------+---------+----------+--------+---------+-----------+-----------+
|    FUTIDX|BANKNIFTY|24-Sep-2020|      0.0|        XX|24900.05| 25270.0| 23371.0| 23743.0|  23743.0|   504222|3051593.27| 1432850|   -54500|31-AUG-2020|       null|
|    FUTIDX|BANKNIFTY|29-Oct-2020|      0.0|        XX|24937.55| 25315.9| 23391.9| 23770.1|  23770.1|    10630|  64526.57|   91375|    19325|31-AUG-2020|       null|
|    FUTIDX|BANKNIFTY|26-Nov-2020|      0.0|        XX|24989.95|25280.75|23437.15| 23766.2|  23766.2|     1171|   7088.68|   14400|     8400|31-AUG-2020|       null|
|   

In [11]:
import matplotlib.pyplot as plt
import numpy as np
date = np.array(futures_data.select('TIMESTAMP').collect()).tolist()
openPrice = np.array(futures_data.select('OPEN').collect()).tolist()
high = np.array(futures_data.select('HIGH').collect()).tolist()

In [12]:
date= np.reshape(date, (1,-1))
openPrice= np.reshape(openPrice, (1,-1))
high = np.reshape(high, (1,-1))
print(date,openPrice,high)

[['31-AUG-2020' '31-AUG-2020' '31-AUG-2020' ... '12-JUN-2000'
  '12-JUN-2000' '12-JUN-2000']] [['24900.05' '24937.55' '24989.95' ... '1461' '1478' '1480']] [['25270.0' '25315.9' '25280.75' ... '1472' '1478' '1482']]


In [13]:
date= date[0]
openPrice= openPrice[0]
high = high[0]
print(date,openPrice,high)

['31-AUG-2020' '31-AUG-2020' '31-AUG-2020' ... '12-JUN-2000' '12-JUN-2000'
 '12-JUN-2000'] ['24900.05' '24937.55' '24989.95' ... '1461' '1478' '1480'] ['25270.0' '25315.9' '25280.75' ... '1472' '1478' '1482']


In [14]:
print(openPrice)

['24900.05' '24937.55' '24989.95' ... '1461' '1478' '1480']


In [16]:
futures_data.coalesce(1).write.csv('FUTIDX.csv')

In [15]:
def correct_date_format(date_str):
    # Split the date string into day, month, and year
    day, month, year = date_str.split('-')

    # Check if day, month, and year are in correct format
    if len(day) != 2:
        # Manually correct day format
        day = '0' + day if len(day) == 1 else day
    if len(month) != 3:
        # Manually correct month format
        month = month[:3]
    if len(year) != 4:
        # Manually correct year format
        year = '20' + year if len(year) == 2 else year

    # Return the corrected date string
    return f"{day}-{month}-{year}"

# Create a new list of corrected dates
corrected_dates = [correct_date_format(date_str) for date_str in date[0]]

print(corrected_dates)

ValueError: not enough values to unpack (expected 3, got 1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from dateutil import parser

date_objects = [datetime.strptime(d, '%d-%b-%Y') for d in corrected_dates]
# date_objects = date_objects.tolist()
# Plotting
plt.figure(figsize=(10,20))
plt.plot(date_objects, openPrice)
plt.title('Open Prices Over Time')
plt.xlabel('Date')
plt.ylabel('Price')
plt.xticks(date_objects, dates, rotation=45)  # Set the tick labels as dates with rotation
plt.grid(True)
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
print((date_objects[100]))

In [ ]:
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler


# Convert string columns to numeric types
futures_data = futures_data.withColumn('OPEN', futures_data['OPEN'].cast('double'))
futures_data = futures_data.withColumn('HIGH', futures_data['HIGH'].cast('double'))
futures_data = futures_data.withColumn('LOW', futures_data['LOW'].cast('double'))
futures_data = futures_data.withColumn('CLOSE', futures_data['CLOSE'].cast('double'))
futures_data = futures_data.withColumn('SETTLE_PR', futures_data['SETTLE_PR'].cast('double')) # Assuming SETTLE_PR is the label column

# Now you can proceed with the VectorAssembler and model training
feature_columns = ['OPEN', 'HIGH', 'LOW', 'CLOSE']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(futures_data)

# Define the neural network architecture
layers = [len(feature_columns), 64, 32, 1]  # Input size, hidden layers, output size
mlp = MultilayerPerceptronClassifier(layers=layers, seed=1, featuresCol="features", labelCol="SETTLE_PR") # Use SETTLE_PR as labelCol

# Split data into training and test sets
(training_data, test_data) = data.randomSplit([0.8, 0.2])

# Train the neural network model
model = mlp.fit(training_data)

# Make predictions on the test data
predictions = model.transform(test_data)


In [ ]:
# Evaluate the model
evaluator = RegressionEvaluator(labelCol="OPTION_PRICE", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
futures_data.printSchema()